In [1]:
from vllm.v1.engine.core_client import EngineCoreClient
EngineCoreClient.make_client??

Signature:
EngineCoreClient.make_client(
    multiprocess_mode: bool,
    asyncio_mode: bool,
    vllm_config: vllm.config.vllm.VllmConfig,
    executor_class: type[vllm.v1.executor.abstract.Executor],
    log_stats: bool,
) -> 'EngineCoreClient'
Docstring: <no docstring>
Source:   
    @staticmethod
    def make_client(
        multiprocess_mode: bool,
        asyncio_mode: bool,
        vllm_config: VllmConfig,
        executor_class: type[Executor],
        log_stats: bool,
    ) -> "EngineCoreClient":
        # TODO: support this for debugging purposes.
        if asyncio_mode and not multiprocess_mode:
            raise NotImplementedError(
                "Running EngineCore in asyncio without multiprocessing "
                "is not currently supported."
            )

        if multiprocess_mode and asyncio_mode:
            return EngineCoreClient.make_async_mp_client(
                vllm_config, executor_class, log_stats
            )

        if multiprocess_mode and not 

In [2]:
# vLLM can run in two modes:

# In-process (InprocClient) - everything in one process, direct function calls
# Multi-process (SyncMPClient) - EngineCore runs in separate process, communicates via IPC

# The "client" pattern lets them use the same interface for both:
# if multiprocess_mode:
#     return SyncMPClient(...)   # talks to EngineCore via IPC
# else:
#     return InprocClient(...)   # talks to EngineCore directly

# So "client" doesn't mean there's a remote server. It means "the thing that talks to EngineCore" - whether that's a direct call or over a socket.
# The "server" is EngineCore itself. InprocClient just wraps it directly without any network/IPC.

In [3]:
from vllm.v1.engine.core_client import InprocClient
InprocClient??

Init signature: InprocClient(*args, **kwargs)
Source:        
class InprocClient(EngineCoreClient):
    """
    InprocClient: client for in-process EngineCore. Intended
    for use in LLMEngine for V0-style add_request() and step()
        EngineCore setup in this process (no busy loop).

        * pushes EngineCoreRequest directly into the EngineCore
        * pulls EngineCoreOutputs by stepping the EngineCore
    """

    def __init__(self, *args, **kwargs):
        self.engine_core = EngineCore(*args, **kwargs)

    def get_output(self) -> EngineCoreOutputs:
        outputs, _ = self.engine_core.step_fn()
        return outputs and outputs.get(0) or EngineCoreOutputs()

    def get_supported_tasks(self) -> tuple[SupportedTask, ...]:
        return self.engine_core.get_supported_tasks()

    def add_request(self, request: EngineCoreRequest) -> None:
        req, request_wave = self.engine_core.preprocess_add_request(request)
        self.engine_core.add_request(req, request_wave)

  

In [4]:
# come on client by itself is wrapper around EngineCore
# def __init__(self, *args, **kwargs):
#     self.engine_core = EngineCore(*args, **kwargs)

from vllm.v1.engine.core import EngineCore
EngineCore??

Init signature:
EngineCore(
    vllm_config: vllm.config.vllm.VllmConfig,
    executor_class: type[vllm.v1.executor.abstract.Executor],
    log_stats: bool,
    executor_fail_callback: collections.abc.Callable | None = None,
)
Source:        
class EngineCore:
    """Inner loop of vLLM's Engine."""

    def __init__(
        self,
        vllm_config: VllmConfig,
        executor_class: type[Executor],
        log_stats: bool,
        executor_fail_callback: Callable | None = None,
    ):
        # plugins need to be loaded at the engine/scheduler level too
        from vllm.plugins import load_general_plugins

        load_general_plugins()

        self.vllm_config = vllm_config
        if vllm_config.parallel_config.data_parallel_rank == 0:
            logger.info(
                "Initializing a V1 LLM engine (v%s) with config: %s",
                VLLM_VERSION,
                vllm_config,
            )

        self.log_stats = log_stats

        # Setup Model.
        self.model